In [45]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, accuracy_score, classification_report, confusion_matrix


In [46]:
CSV = "HA Data.csv"
try:
    df = pd.read_csv(CSV, low_memory=False)
    print("Loaded:", CSV, "| rows:", len(df), "cols:", len(df.columns))
except FileNotFoundError:
    raise FileNotFoundError(f"File {CSV} not found. Put HA Data.csv in notebook folder.")


Loaded: HA Data.csv | rows: 66587 cols: 49


In [47]:
print("Columns (sample):", list(df.columns)[:30])
print("\nNulls by column (top 10):")
print(df.isnull().mean().sort_values(ascending=False).head(10).round(3))
print("\nreadmitted present?:", 'readmitted' in df.columns)


Columns (sample): ['index', 'encounter_id', 'patient_id', 'race', 'gender', 'age', 'weight', 'time_in_hospital', 'medical_specialty', 'num_lab_procedures', 'num_procedures', 'num_medications', 'number_outpatient', 'number_emergency', 'number_inpatient', 'diag_1', 'diag_2', 'diag_3', 'diag_4', 'diag_5', 'number_diagnoses', 'X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8', 'X9']

Nulls by column (top 10):
index    0.0
X5       0.0
X7       0.0
X8       0.0
X9       0.0
X10      0.0
X11      0.0
X12      0.0
X13      0.0
X14      0.0
dtype: float64

readmitted present?: True


In [48]:
if 'readmitted' not in df.columns:
    raise RuntimeError("Dataset missing required 'readmitted' column.")
df = df.dropna(subset=['readmitted']).copy()
def _norm_read(x):
    try:
        return int(x)
    except:
        s=str(x).strip().lower()
        return 1 if s.startswith('1') or s in ['yes','y','true'] else 0
df['readmitted'] = df['readmitted'].apply(_norm_read).astype(int)

for c in ['admission_date','discharge_date','followup_appointment_date']:
    if c in df.columns:
        df[c] = pd.to_datetime(df[c], errors='coerce')

df = df.drop_duplicates().reset_index(drop=True)
print("After cleaning, rows:", len(df))


After cleaning, rows: 48911


In [49]:
if 'time_in_hospital' in df.columns:
    df['time_in_hospital'] = pd.to_numeric(df['time_in_hospital'], errors='coerce')
elif 'admission_date' in df.columns and 'discharge_date' in df.columns:
    df['time_in_hospital'] = (pd.to_datetime(df['discharge_date'], errors='coerce') - pd.to_datetime(df['admission_date'], errors='coerce')).dt.days
else:
    df['time_in_hospital'] = np.nan
df['time_in_hospital'] = df['time_in_hospital'].clip(lower=0)

diag_cols = [c for c in df.columns if c.lower().startswith('diag_')]
if 'number_diagnoses' not in df.columns:
    df['number_diagnoses'] = df[diag_cols].notna().sum(axis=1) if diag_cols else 0

if 'diabetesMed' not in df.columns:
    df['diabetesMed'] = 'No'
else:
    df['diabetesMed'] = df['diabetesMed'].fillna('No').astype(str)
df['med_change_flag'] = df.get('change', 'No').fillna('No').astype(str).str.lower() == 'yes'

def _los_pts(x):
    if pd.isna(x): return 0
    try:
        x=int(x)
    except:
        return 0
    return 0 if x<=1 else (1 if x<=3 else (2 if x<=6 else 3))
df['L_pts'] = df['time_in_hospital'].apply(_los_pts)
if 'admission_type' in df.columns:
    df['A_pts'] = np.where(df['admission_type'].astype(str).str.lower()=='emergency', 3, 0)
else:
    df['A_pts'] = 0
df['C_pts'] = pd.cut(df['number_diagnoses'].fillna(0).astype(int), bins=[-1,1,3,6,100], labels=[0,1,2,3]).astype(int)
if 'number_emergency' in df.columns:
    df['E_pts'] = pd.cut(df['number_emergency'].fillna(0).astype(int), bins=[-1,0,1,3,100], labels=[0,1,2,3]).astype(int)
else:
    df['E_pts'] = 0
df['lace_score'] = df[['L_pts','A_pts','C_pts','E_pts']].sum(axis=1)

if 'age' in df.columns:
    df['age'] = pd.to_numeric(df['age'], errors='coerce')
    df['age_group'] = pd.cut(df['age'], bins=[0,30,50,65,80,200], labels=['<30','30-49','50-64','65-79','80+'])
else:
    df['age_group'] = 'unknown'

print("Feature engineering done. Sample:")
print(df[['time_in_hospital','number_diagnoses','lace_score']].head(3))


Feature engineering done. Sample:
   time_in_hospital  number_diagnoses  lace_score
0                 4                 9           6
1                 3                 6           3
2                 6                 9           5


## BASIC QUESTIONS (B1–B10) — compact answers


In [50]:
if 'age_group' in df.columns:
    b1 = df.groupby('age_group')['readmitted'].agg(n='size', readmit_rate='mean').reset_index()
    print("B1 — Readmit by age_group (compact):")
    print(b1.to_string(index=False))
else:
    print("B1 skipped — 'age' not present.")


B1 — Readmit by age_group (compact):
age_group  n  readmit_rate
      <30  0           NaN
    30-49  0           NaN
    50-64  0           NaN
    65-79  0           NaN
      80+  0           NaN


In [51]:
if 'medical_specialty' in df.columns and df['time_in_hospital'].notna().any():
    b2 = df.groupby('medical_specialty')['time_in_hospital'].median().sort_values(ascending=False).head(10)
    print("B2 — Median LOS by specialty (top10):")
    print(b2.to_string())
else:
    print("B2 skipped — 'medical_specialty' or LOS missing.")


B2 — Median LOS by specialty (top10):
medical_specialty
Pediatrics-Pulmonology               10.0
PhysicalMedicineandRehabilitation     9.0
Rheumatology                          9.0
OutreachServices                      8.5
Pathology                             8.0
Psychiatry-Addictive                  8.0
Proctology                            7.0
Surgery-Colon&Rectal                  6.5
Psychology                            6.0
Surgery-Cardiovascular/Thoracic       6.0


In [52]:
if 'number_emergency' in df.columns:
    corr = df['number_emergency'].corr(df['readmitted'])
    print(f"B3 — Corr(number_emergency, readmitted) = {corr:.3f}")
else:
    print("B3 skipped — 'number_emergency' missing.")


B3 — Corr(number_emergency, readmitted) = 0.109


In [53]:
if 'diabetesMed' in df.columns:
    mask = df['diabetesMed'].astype(str).str.lower().isin(['yes','y','true','1'])
    if mask.sum()>0:
        print("B4 — Readmit among diabetesMed =", df.loc[mask,'readmitted'].mean().round(3), "n=", mask.sum())
    else:
        print("B4: no diabetesMed==Yes rows.")
else:
    print("B4 skipped — 'diabetesMed' missing.")


B4 — Readmit among diabetesMed = 0.479 n= 37640


In [54]:
if 'med_change_flag' in df.columns:
    b5 = df.groupby('med_change_flag')['readmitted'].agg(n='size', readmit_rate='mean').reset_index()
    print("B5 — med_change_flag readmit rates:")
    print(b5.to_string(index=False))
else:
    print("B5 skipped — med_change_flag missing.")


B5 — med_change_flag readmit rates:
 med_change_flag     n  readmit_rate
           False 48911      0.462248


In [55]:
if 'num_lab_procedures' in df.columns:
    m = df.groupby('readmitted')['num_lab_procedures'].mean().to_dict()
    print("B6 — mean num_lab_procedures: not_readmit =", round(m.get(0, np.nan),3), "readmit =", round(m.get(1, np.nan),3))
else:
    print("B6 skipped — num_lab_procedures missing.")


B6 — mean num_lab_procedures: not_readmit = 42.437 readmit = 43.894


In [56]:
if 'race' in df.columns:
    r = df.groupby('race')['readmitted'].mean().sort_values(ascending=False).head(6) * 100
    print("B7 — Readmission Rate by Race (Top 6):")
    print(r.apply(lambda x: f"{x:.2f}%").to_string())
else:
    print("B7 race skipped — 'race' column not found in dataset.")

if 'gender' in df.columns:
    g = df.groupby('gender')['readmitted'].mean() * 100
    print("\nB7 — Readmission Rate by Gender:")
    print(g.apply(lambda x: f"{x:.2f}%").to_string())
else:
    print("B7 gender skipped — 'gender' column not found in dataset.")

B7 — Readmission Rate by Race (Top 6):
race
Caucasian          47.02%
AfricanAmerican    46.13%
Hispanic           41.13%
Other              40.58%
Asian              37.00%
?                  31.51%

B7 — Readmission Rate by Gender:
gender
Female             46.83%
Male               45.52%
Unknown/Invalid     0.00%


In [57]:
if 'weight' in df.columns:
    w = pd.to_numeric(df['weight'], errors='coerce').dropna()
    if len(w) < 10 or w.nunique() < 4:
        print("B8 skipped — insufficient weight data for quartiles.")
    else:
        df['_weight_bin'] = pd.qcut(df['weight'].dropna(), 4, labels=['Q1','Q2','Q3','Q4'])
        out = df.dropna(subset=['_weight_bin']).groupby('_weight_bin')['readmitted'].mean()
        print("B8 — readmit by weight quartile:")
        print(out.to_string())
else:
    print("B8 skipped — 'weight' missing.")


B8 skipped — insufficient weight data for quartiles.


In [58]:
if 'num_medications' in df.columns and df['time_in_hospital'].notna().any():
    corr = df['num_medications'].corr(df['time_in_hospital'])
    print(f"B9 — Corr(num_medications, LOS) = {corr:.3f}")
else:
    print("B9 skipped — columns missing.")


B9 — Corr(num_medications, LOS) = 0.465


In [59]:
if 'number_outpatient' in df.columns:
    corr = df['number_outpatient'].corr(df['readmitted'])
    print(f"B10 — Corr(number_outpatient, readmitted) = {corr:.3f}")
else:
    print("B10 skipped — 'number_outpatient' missing.")


B10 — Corr(number_outpatient, readmitted) = 0.083


## MEDIUM QUESTIONS (M1–M10) — compact answers


In [60]:
if 'admission_date' in df.columns:
    monthly = df.groupby(df['admission_date'].dt.to_period('M'))['readmitted'].mean()
    print("M1 — recent monthly readmit (last 6 periods):")
    print(monthly.tail(6).round(3).to_string())
else:
    print("M1 skipped — 'admission_date' missing.")


M1 skipped — 'admission_date' missing.


In [75]:
if 'number_diagnoses' in df.columns:
    m2 = df.groupby('number_diagnoses').agg(
        n=('encounter_id','count'),
        median_los=('time_in_hospital','median'),
        readmit_rate=('readmitted','mean')
    ).reset_index().sort_values('n', ascending=False).head(10)


    m2['readmit_rate'] = (m2['readmit_rate'] * 100).apply(lambda x: f"{x:.2f}%")

    print("M2 — Sample by Number of Diagnoses (Top 10):")
    print(m2.to_string(index=False))
else:
    print("M2 skipped — 'number_diagnoses' column not found in dataset.")


M2 — Sample by Number of Diagnoses (Top 10):
 number_diagnoses     n  median_los readmit_rate
                9 23832         4.0       50.73%
                5  5484         3.0       36.03%
                8  5116         4.0       47.60%
                7  4942         3.0       46.46%
                6  4855         3.0       44.30%
                4  2657         3.0       37.15%
                3  1378         2.0       34.83%
                2   493         2.0       29.82%
                1   102         2.0       22.55%
               16    19         5.0       36.84%


In [61]:
cols = [c for c in ['num_lab_procedures','num_procedures'] if c in df.columns]
if cols and 'race' in df.columns:
    m4 = df.groupby('race')[cols].mean().head(8)
    print("M4 — avg procedure counts by race (sample):")
    print(m4.round(2).to_string())
else:
    print("M4 skipped — required columns missing.")


M4 — avg procedure counts by race (sample):
                 num_lab_procedures  num_procedures
race                                               
?                             44.36            1.23
AfricanAmerican               44.22            1.22
Asian                         41.53            1.41
Caucasian                     42.84            1.37
Hispanic                      41.66            1.08
Other                         43.55            1.52


In [62]:
x_cols = [c for c in df.columns if c.startswith('X') and c[1:].isdigit()]
if x_cols and 'medical_specialty' in df.columns:
    m7 = df.groupby('medical_specialty')[x_cols].mean().head(8)
    print("M7 — medication indicators by specialty (sample):")
    print(m7.round(2).to_string())
else:
    print("M7 skipped — Xcols or medical_specialty missing.")


M7 — medication indicators by specialty (sample):
Empty DataFrame
Columns: []
Index: [?, AllergyandImmunology, Anesthesiology, Anesthesiology-Pediatric, Cardiology, Cardiology-Pediatric, DCPTEAM, Dentistry]


In [63]:
if 'weight' in df.columns and 'primary_dx' in df.columns:
    wnum = pd.to_numeric(df['weight'], errors='coerce')
    if wnum.dropna().shape[0] < 10:
        print("M8 skipped — insufficient weight data.")
    else:
        m8 = df.dropna(subset=['weight']).groupby('primary_dx').agg(n=('encounter_id','count'), mean_w=('weight','mean'), readmit_rate=('readmitted','mean')).sort_values('n', ascending=False).head(10)
        print("M8 — sample primary_dx with weight stats:")
        print(m8.round(2).to_string())
else:
    print("M8 skipped — weight or primary_dx missing.")


M8 skipped — weight or primary_dx missing.


In [64]:
if 'diabetesMed' in df.columns and 'med_change_flag' in df.columns:
    dmask = df['diabetesMed'].astype(str).str.lower().isin(['yes','1','y','true'])
    if dmask.sum()>0:
        m9 = df[dmask].groupby('med_change_flag')['readmitted'].agg(n='size', readmit_rate='mean')
        print("M9 — diabetic readmit by med_change_flag:")
        print(m9.to_string())
    else:
        print("M9 skipped — no diabetesMed==Yes rows.")
else:
    print("M9 skipped — required columns missing.")


M9 — diabetic readmit by med_change_flag:
                     n  readmit_rate
med_change_flag                     
False            37640      0.479384


In [65]:
scol = next((c for c in ['patient_satisfaction','satisfaction','satis_score'] if c in df.columns), None)
if scol:
    qlabels = pd.qcut(df[scol].rank(method='first'), 4, labels=['Q1','Q2','Q3','Q4'])
    m10 = df.groupby(qlabels)['readmitted'].mean()
    print("M10 — readmit by satisfaction quartile:")
    print(m10.round(3).to_string())
else:
    print("M10 skipped — satisfaction column missing.")


M10 skipped — satisfaction column missing.


## ADVANCED QUESTIONS (A1–A10) — compact answers & models


In [66]:
candidate_features = [
    'time_in_hospital','num_lab_procedures','num_procedures','num_medications',
    'number_outpatient','number_emergency','number_inpatient','number_diagnoses','lace_score',
    'age_group','gender','race','diabetesMed','med_change_flag'
]
features = [c for c in candidate_features if c in df.columns]
df_model = df.dropna(subset=features + ['readmitted']).copy()
print("A1 — modeling rows:", len(df_model), "| features used:", features)


A1 — modeling rows: 0 | features used: ['time_in_hospital', 'num_lab_procedures', 'num_procedures', 'num_medications', 'number_outpatient', 'number_emergency', 'number_inpatient', 'number_diagnoses', 'lace_score', 'age_group', 'gender', 'race', 'diabetesMed', 'med_change_flag']


In [67]:
if all(c in df.columns for c in ['number_outpatient','number_emergency','number_inpatient']):
    df['util_sum'] = df[['number_outpatient','number_emergency','number_inpatient']].fillna(0).sum(axis=1)
    df['util_bucket'] = pd.qcut(df['util_sum'].rank(method='first'), 3, labels=['Low','Med','High'])
    print("A2 — readmit by util_bucket:")
    print(df.groupby('util_bucket')['readmitted'].mean().round(3).to_string())
else:
    print("A2 skipped — utilization columns missing.")


A2 — readmit by util_bucket:
util_bucket
Low     0.362
Med     0.413
High    0.611


In [68]:
if 'cost' in df.columns:
    avg_cost = df.loc[df['readmitted']==1,'cost'].mean()
    total_readmits = df['readmitted'].sum()
    print(f"A3 — avg cost per readmit: {avg_cost:.2f}, total readmits: {total_readmits}")
else:
    print("A3 skipped — 'cost' not present.")


A3 skipped — 'cost' not present.


In [69]:
for col in ['race','payer','zip']:
    if col in df.columns:
        out = df.groupby(col)['readmitted'].agg(n='size', readmit_rate='mean').sort_values('readmit_rate', ascending=False).head(5)
        print(f"A4 — top by {col}:")
        print(out.to_string())
    else:
        print(f"A4 — {col} missing; skipped.")


A4 — top by race:
                     n  readmit_rate
race                                
Caucasian        36637      0.470235
AfricanAmerican   9136      0.461252
Hispanic           970      0.411340
Other              754      0.405836
Asian              300      0.370000
A4 — payer missing; skipped.
A4 — zip missing; skipped.


In [70]:
if 'followup_appointment_date' in df.columns and 'discharge_date' in df.columns:
    df['days_to_followup'] = (pd.to_datetime(df['followup_appointment_date'], errors='coerce') - pd.to_datetime(df['discharge_date'], errors='coerce')).dt.days
    df['followup_7d'] = df['days_to_followup'].between(0,7)
    print("A5 — readmit by followup_7d:")
    print(df.groupby('followup_7d')['readmitted'].agg(n='size', readmit_rate='mean').to_string())
else:
    print("A5 skipped — followup or discharge dates missing.")


A5 skipped — followup or discharge dates missing.


In [71]:
rec = [
 "1) Clinical review of top-200 predicted high-risk patients.",
 "2) Pilot: pharmacist callback + 7-day follow-up for top-risk group.",
 "3) Capture SDoH & discharge disposition in EMR.",
 "4) Retrain monthly and add local explanations (SHAP).",
 "5) Partner with finance to calculate ROI per avoided readmission."
]
print("A10 — Recommendations:")
for r in rec:
    print(r)


A10 — Recommendations:
1) Clinical review of top-200 predicted high-risk patients.
2) Pilot: pharmacist callback + 7-day follow-up for top-risk group.
3) Capture SDoH & discharge disposition in EMR.
4) Retrain monthly and add local explanations (SHAP).
5) Partner with finance to calculate ROI per avoided readmission.


In [72]:
print("FINAL SUMMARY (compact):")
print("-", "Rows used:", len(df))
print("-", "Overall readmit rate:", f"{df['readmitted'].mean():.2%}")
if 'rf_pipe' in globals() or 'rf_pipe' in locals():
    print("-", "RF AUC (test):", round(rf_auc,3))
    print("-", "Top risk export:", EXPORT)
else:
    print("-", "No model trained due to limited data.")
print("\nNext steps (compact): clinical review of top list; pilot outreach; capture SDoH & discharge details.")


FINAL SUMMARY (compact):
- Rows used: 48911
- Overall readmit rate: 46.22%
- No model trained due to limited data.

Next steps (compact): clinical review of top list; pilot outreach; capture SDoH & discharge details.


In [73]:
KPIS = {
    'rows': len(df),
    'readmit_rate': df['readmitted'].mean()
}
pd.DataFrame([KPIS]).to_csv("kpi_summary.csv", index=False)
print("Saved KPI summary to kpi_summary.csv — notebook complete.")


Saved KPI summary to kpi_summary.csv — notebook complete.
